In [13]:
from kafka import KafkaConsumer, TopicPartition, KafkaProducer
import pickle
import json
import pandas as pd
import pymongo
import re
import time
from bson import json_util

In [14]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["credit_card_transaction"]
mycol = mydb['predict_result']

In [15]:
model = pickle.load(open("model.pkl", "rb"))

In [16]:
producer = KafkaProducer(bootstrap_servers='localhost:9092', value_serializer=lambda v: json.dumps(v).encode('utf-8'))

In [17]:
def extract_record_from_message(message):
    # Tìm phần "record"
    record_start = message.find('"record": {')
    if record_start == -1:
        return None  # Không tìm thấy phần record
    
    # Đếm dấu ngoặc để tách phần record chính xác
    opening_braces = 1  # Đã tìm thấy dấu mở ngoặc đầu tiên '{'
    record_str = message[record_start + len('"record": {'):]
    index = 0
    
    for i, char in enumerate(record_str):
        if char == '{':
            opening_braces += 1
        elif char == '}':
            opening_braces -= 1
        
        if opening_braces == 0:
            index = i
            break
    
    # Tách phần record
    record_str = "{" + record_str[:index] + "}"
    
    try:
        # Chuyển đổi phần record thành dictionary
        record_dict = json.loads(record_str)
        return record_dict
    except json.JSONDecodeError as e:
        print("JSON Decode Error:", e)
        return None

In [ ]:
result = {0: "Don't Fraud",
          1: "Is Fraud"}

In [24]:
consumer = KafkaConsumer(
    'input',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='earliest'  # Start reading from the earliest offset
)   

for msg in consumer:

    message = msg.value.decode('utf-8')
    data = json.loads(message)
    dict_data = extract_record_from_message(data)
    df = pd.DataFrame([dict_data])
        
        # Xử lý dữ liệu
    time_transaction = df['trans_date_trans_time'][0]
    df = df.drop(columns=['Unnamed: 0', 'trans_date_trans_time', 'trans_num', 'unix_time', 'dob', 'first', 'last'])
    df['merch_zipcode'] = df['merch_zipcode'].fillna(-1)
        # Dự đoán
    predict_result = model.predict(df)        
        # Lưu vào MongoDB và Kafka
    save_data = {
        "transaction": time_transaction,
        "predict_result": result[predict_result[0]]
    }

    message_json = json.dumps(save_data)
    mycol.insert_one(save_data)
 
        # Gửi đến Kafka
    producer.send("output", message_json)
        
        # In kết quả
    print("Time Sent:", time.strftime('%Y-%m-%d %H:%M:%S'))
    print("Predictions:", predict_result)
        

<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:52
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-08-24 09:25:53
Predictions: [0]
<class 'str'>
<class 'numpy.ndarray'>
Time Sent: 2024-

KeyboardInterrupt: 